## Dependencies

In [1]:
import glob
import warnings
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalAveragePooling1D, SpatialDropout1D, Concatenate, Add, Average
from googleqa_utilityscript import *
from transformers import BertConfig, BertTokenizer, TFBertModel


SEED = 0
seed_everything(SEED)
warnings.filterwarnings("ignore")

## Load data

In [2]:
MODULE_BASE_PATH = '/kaggle/input/bert-base-uncased-huggingface/bert-base-uncased'
MODULE_PATH = MODULE_BASE_PATH + '/bert-base-uncased-tf_model.h5'
VOCAB_PATH = MODULE_BASE_PATH + '/bert-base-uncased-vocab.txt'
CONF_PATH = MODULE_BASE_PATH + '/bert-base-uncased-config.json'
model_path_list = glob.glob('/kaggle/input/148-googleq-a-train-3fold-bert-base-unc-qa3/' + '*1.h5')
model_path_list += glob.glob('/kaggle/input/148-googleq-a-train-3fold-bert-base-unc-qa3/' + '*2.h5')
model_path_list += glob.glob('/kaggle/input/148-googleq-a-train-3fold-bert-base-unc-qa3/' + '*3.h5')
model_path_list.sort()
print('Models to predict:')
print(*model_path_list, sep = "\n")

MODULE_BASE_PATH_CASED = '/kaggle/input/bert-base-cased-huggingface/bert-base-cased'
MODULE_PATH_CASED = MODULE_BASE_PATH_CASED + '/bert-base-cased-tf_model.h5'
VOCAB_PATH_CASED = MODULE_BASE_PATH_CASED + '/bert-base-cased-vocab.txt'
CONF_PATH_CASED = MODULE_BASE_PATH_CASED + '/bert-base-cased-config.json'
model_path_list_cased = glob.glob('/kaggle/input/154-googleq-a-train-3fold-bert-base-cased-qa3/' + '*1.h5')
model_path_list_cased += glob.glob('/kaggle/input/154-googleq-a-train-3fold-bert-base-cased-qa3/' + '*2.h5')
model_path_list_cased += glob.glob('/kaggle/input/154-googleq-a-train-3fold-bert-base-cased-qa3/' + '*3.h5')
model_path_list_cased.sort()
print('Models to predict (cased):')
print(*model_path_list_cased, sep = "\n")

test = pd.read_csv('/kaggle/input/google-quest-challenge/test.csv')

print('Test samples: %s' % len(test))
display(test.head())

Models to predict:
/kaggle/input/148-googleq-a-train-3fold-bert-base-unc-qa3/model_fold_1.h5
/kaggle/input/148-googleq-a-train-3fold-bert-base-unc-qa3/model_fold_2.h5
/kaggle/input/148-googleq-a-train-3fold-bert-base-unc-qa3/model_fold_3.h5
Models to predict (cased):
/kaggle/input/154-googleq-a-train-3fold-bert-base-cased-qa3/model_fold_1.h5
/kaggle/input/154-googleq-a-train-3fold-bert-base-cased-qa3/model_fold_2.h5
/kaggle/input/154-googleq-a-train-3fold-bert-base-cased-qa3/model_fold_3.h5
Test samples: 476


,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,host
0,39,Will leaving corpses lying around upset my pri...,I see questions/information online about how t...,Dylan,https://gaming.stackexchange.com/users/64471,There is no consequence for leaving corpses an...,Nelson868,https://gaming.stackexchange.com/users/97324,http://gaming.stackexchange.com/questions/1979...,CULTURE,gaming.stackexchange.com
1,46,Url link to feature image in the portfolio,I am new to Wordpress. i have issue with Featu...,Anu,https://wordpress.stackexchange.com/users/72927,I think it is possible with custom fields.\n\n...,Irina,https://wordpress.stackexchange.com/users/27233,http://wordpress.stackexchange.com/questions/1...,TECHNOLOGY,wordpress.stackexchange.com
2,70,"Is accuracy, recoil or bullet spread affected ...","To experiment I started a bot game, toggled in...",Konsta,https://gaming.stackexchange.com/users/37545,You do not have armour in the screenshots. Thi...,Damon Smithies,https://gaming.stackexchange.com/users/70641,http://gaming.stackexchange.com/questions/2154...,CULTURE,gaming.stackexchange.com
3,132,Suddenly got an I/O error from my external HDD,I have used my Raspberry Pi as a torrent-serve...,robbannn,https://raspberrypi.stackexchange.com/users/17341,Your Western Digital hard drive is disappearin...,HeatfanJohn,https://raspberrypi.stackexchange.com/users/1311,http://raspberrypi.stackexchange.com/questions...,TECHNOLOGY,raspberrypi.stackexchange.com
4,200,Passenger Name - Flight Booking Passenger only...,I have bought Delhi-London return flights for ...,Amit,https://travel.stackexchange.com/users/29089,I called two persons who work for Saudia (tick...,Nean Der Thal,https://travel.stackexchange.com/users/10051,http://travel.stackexchange.com/questions/4704...,CULTURE,travel.stackexchange.com


In [3]:
question_target_cols = ['question_asker_intent_understanding','question_body_critical', 'question_conversational', 
                        'question_expect_short_answer', 'question_fact_seeking', 'question_has_commonly_accepted_answer',
                        'question_interestingness_others', 'question_interestingness_self', 'question_multi_intent', 
                        'question_not_really_a_question', 'question_opinion_seeking', 'question_type_choice',
                        'question_type_compare', 'question_type_consequence', 'question_type_definition', 
                        'question_type_entity', 'question_type_instructions', 'question_type_procedure',
                        'question_type_reason_explanation', 'question_type_spelling', 'question_well_written']
answer_target_cols = ['answer_helpful', 'answer_level_of_information', 'answer_plausible', 'answer_relevance',
                      'answer_satisfaction', 'answer_type_instructions', 'answer_type_procedure', 
                      'answer_type_reason_explanation', 'answer_well_written']
target_cols = question_target_cols + answer_target_cols

## Pre-process data

In [4]:
text_features = ['question_title', 'question_body', 'answer']
text_features_question = ['question_title', 'question_body']
text_features_answer = ['question_title', 'answer']

# Model parameters

In [5]:
N_CLASS = len(target_cols)
N_CLASS_QUESTION = len(question_target_cols)
N_CLASS_ANSWER = len(answer_target_cols)
MAX_SEQUENCE_LENGTH = 512
MAX_LENGTH_TITLE = 30
MAX_LENGTH_QUESTION = 479
MAX_LENGTH_ANSWER = 479

## Test set

In [6]:
tokenizer = BertTokenizer.from_pretrained(VOCAB_PATH, do_lower_case=True)

# Test features
X_test = compute_input_arays_2(test, text_features_question, tokenizer, MAX_SEQUENCE_LENGTH, 
                               MAX_LENGTH_TITLE, MAX_LENGTH_QUESTION, ignore_first_sep=False)\
        + compute_input_arays_2(test, text_features_answer, tokenizer, MAX_SEQUENCE_LENGTH, 
                                MAX_LENGTH_TITLE, MAX_LENGTH_ANSWER, ignore_first_sep=False)

tokenizer_cased = BertTokenizer.from_pretrained(VOCAB_PATH_CASED, do_lower_case=False)

# Test features
X_test_cased = compute_input_arays_2(test, text_features_question, tokenizer_cased, MAX_SEQUENCE_LENGTH, 
                                     MAX_LENGTH_TITLE, MAX_LENGTH_QUESTION, ignore_first_sep=False)\
                + compute_input_arays_2(test, text_features_answer, tokenizer_cased, MAX_SEQUENCE_LENGTH, 
                                        MAX_LENGTH_TITLE, MAX_LENGTH_ANSWER, ignore_first_sep=False)

# Model

In [7]:
module_config = BertConfig.from_pretrained(CONF_PATH)
module_config.output_hidden_states=False

def model_fn():
    input_word_ids_question = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_word_ids_question')
    input_masks_question = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_masks_question')
    segment_ids_question = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='segment_ids_question')

    input_word_ids_answer = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_word_ids_answer')
    input_masks_answer = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_masks_answer')
    segment_ids_answer = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='segment_ids_answer')

    module_model = TFBertModel.from_pretrained(MODULE_PATH, config=module_config)
    sequence_output_question, pooled_output_question = module_model([input_word_ids_question, input_masks_question, segment_ids_question])
    sequence_output_answer, pooled_output_answer = module_model([input_word_ids_answer, input_masks_answer, segment_ids_answer])
    
    x_question = GlobalAveragePooling1D()(sequence_output_question)
    x_question = Dropout(0.2)(x_question)
    output_question = Dense(N_CLASS_QUESTION, kernel_initializer='glorot_uniform', activation="sigmoid", name="outputquestion")(x_question)
    
    x_answer = GlobalAveragePooling1D()(sequence_output_answer)
    x_answer = Dropout(0.2)(x_answer)
    output_answer = Dense(N_CLASS_ANSWER, kernel_initializer='glorot_uniform', activation="sigmoid", name="output_answer")(x_answer)
    
    output = Concatenate()([output_question, output_answer])

    model = Model(inputs=[input_word_ids_question, input_masks_question, segment_ids_question, 
                          input_word_ids_answer, input_masks_answer, segment_ids_answer], outputs=output)

    return model


module_config_cased = BertConfig.from_pretrained(CONF_PATH_CASED)
module_config_cased.output_hidden_states=False

def model_fn_cased():
    input_word_ids_question = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_word_ids_question')
    input_masks_question = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_masks_question')
    segment_ids_question = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='segment_ids_question')

    input_word_ids_answer = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_word_ids_answer')
    input_masks_answer = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_masks_answer')
    segment_ids_answer = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='segment_ids_answer')

    module_model = TFBertModel.from_pretrained(MODULE_PATH_CASED, config=module_config_cased)
    sequence_output_question, pooled_output_question = module_model([input_word_ids_question, input_masks_question, segment_ids_question])
    sequence_output_answer, pooled_output_answer = module_model([input_word_ids_answer, input_masks_answer, segment_ids_answer])
    
    x_question = GlobalAveragePooling1D()(sequence_output_question)
    x_question = Dropout(0.2)(x_question)
    output_question = Dense(N_CLASS_QUESTION, kernel_initializer='glorot_uniform', activation="sigmoid", name="outputquestion")(x_question)
    
    x_answer = GlobalAveragePooling1D()(sequence_output_answer)
    x_answer = Dropout(0.2)(x_answer)
    output_answer = Dense(N_CLASS_ANSWER, kernel_initializer='glorot_uniform', activation="sigmoid", name="output_answer")(x_answer)
    
    output = Concatenate()([output_question, output_answer])

    model = Model(inputs=[input_word_ids_question, input_masks_question, segment_ids_question, 
                          input_word_ids_answer, input_masks_answer, segment_ids_answer], outputs=output)

    return model

# Make predictions

In [8]:
Y_test = np.zeros((len(test), N_CLASS))

for model_path in model_path_list:
    model = model_fn()
    model.load_weights(model_path)
    Y_test += (model.predict(X_test) / 2) / len(model_path_list)

for model_path in model_path_list_cased:
    model = model_fn_cased()
    model.load_weights(model_path)
    Y_test += (model.predict(X_test_cased) / 2) / len(model_path_list_cased)

In [9]:
submission = pd.read_csv('/kaggle/input/google-quest-challenge/sample_submission.csv')
submission[target_cols] = Y_test
submission.to_csv("submission.csv", index=False)
display(submission.head())
display(submission.describe())

,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,39,0.949828,0.641797,0.172857,0.595219,0.588635,0.605312,0.670538,0.642700,0.453165,...,0.913196,0.902582,0.633543,0.956514,0.976348,0.840770,0.012691,0.019618,0.852684,0.925258
1,46,0.876132,0.537857,0.007394,0.781713,0.773838,0.930487,0.533189,0.441254,0.069204,...,0.662994,0.946800,0.641550,0.969258,0.979552,0.880907,0.910581,0.131287,0.109947,0.893126
2,70,0.919916,0.713449,0.040431,0.767905,0.854708,0.906333,0.599655,0.516540,0.099556,...,0.892921,0.943351,0.631855,0.966067,0.976687,0.870584,0.029476,0.031658,0.874699,0.922394
3,132,0.898911,0.502463,0.011216,0.741535,0.731027,0.892101,0.538447,0.428563,0.105636,...,0.741689,0.954772,0.715955,0.963970,0.979647,0.910173,0.716152,0.155085,0.684463,0.905122
4,200,0.927207,0.493507,0.034985,0.871419,0.781038,0.906844,0.611951,0.562843,0.078156,...,0.693576,0.916208,0.657983,0.964426,0.972752,0.831446,0.153332,0.102655,0.713699,0.929262


,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
count,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,...,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000
mean,5029.186975,0.906342,0.619730,0.039230,0.753227,0.787409,0.844470,0.576947,0.491373,0.176162,...,0.803064,0.937530,0.670476,0.961607,0.975190,0.874293,0.476576,0.127037,0.556967,0.912894
std,2812.670060,0.039367,0.134970,0.058426,0.083745,0.095320,0.120017,0.050176,0.092005,0.155765,...,0.085648,0.025651,0.046274,0.012332,0.011453,0.039672,0.309024,0.057024,0.268974,0.020762
min,39.000000,0.753257,0.339983,0.004111,0.359761,0.383636,0.198533,0.489029,0.346514,0.012689,...,0.585141,0.783405,0.555627,0.897069,0.891780,0.720185,0.006856,0.010564,0.051596,0.840944
25%,2572.000000,0.880332,0.506429,0.010109,0.710545,0.746320,0.822225,0.536894,0.418976,0.064431,...,0.729446,0.924616,0.639851,0.956271,0.971036,0.853945,0.166272,0.083934,0.343678,0.900504
50%,5093.000000,0.909619,0.599507,0.015939,0.759124,0.799555,0.888729,0.562058,0.457072,0.116340,...,0.797675,0.943138,0.669488,0.963714,0.978114,0.881462,0.515917,0.128414,0.578262,0.914888
75%,7482.000000,0.937583,0.739665,0.034824,0.802703,0.843955,0.917626,0.609481,0.557829,0.255889,...,0.890520,0.956094,0.699171,0.970213,0.982944,0.902853,0.758885,0.167073,0.801580,0.928202
max,9640.000000,0.978414,0.912095,0.437477,0.962363,0.970550,0.972249,0.727505,0.765216,0.826425,...,0.960699,0.975419,0.814317,0.982153,0.990760,0.946712,0.935746,0.294503,0.975515,0.954594
